In [15]:
import torch
import torch.nn as nn

import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

Define loss tracker

In [ ]:
def loss_tracker(loss_plot, loss_value, num):
    pass


In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)

In [17]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512,
                                          shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                     download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse','ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [18]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

def imshow(img):
    img = img /2 +0.5   # unnormalize
    npimg=img.numpy()
    plt.imshow(np.transpose(npimg, (12,0)))
    plt.show()

# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()
# vis.images(images/2 + 0.5)

# show images
#mshow(torchvision.utils.make_grid(images))

# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

truck   dog horse truck


In [19]:
import torchvision.models.vgg as vgg

In [20]:
cfg = [32, 32, 'M', 64, 64, 128, 128, 128, 'M', 256, 256, 256, 512, 512, 512, 'M']  # 13 + 3 VGG16

In [21]:
class VGG(nn.Module):

    def __init__(self, features, num_classes=1000, init_weights=True):
        super(VGG, self).__init__()
        self.features = features     # 아래 make_layers를 통해 만들어진 CNN 구조를 의미
        # self.avgpool = nn.AdaptiveAvgPool2d((7, 7))    데이터 이미지 크기가 7 x 7 보다 작으므로 생략
        self.classifier = nn.Sequential(
            nn.Linear(512 * 4 * 4, 4096),          # 분류하려는 Image 크기에 맞춰 수정 필요
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        # x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):           # 초기화 조건 설정
        for m in self.modules():             # module을 구성하는 layer를 하나씩 진행하면서 layer에 대해...
            if isinstance(m, nn.Conv2d):         # m이 nn.Conv2d인지 아닌지 확인한다.
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)     # Conv2d의 bias를 0으로 한다는 의미
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

In [22]:
vgg16 = VGG(vgg.make_layers(cfg), 10, True).to(device)

In [9]:
a=torch.Tensor(1,3,32,32).to(device)       # 모델이 정상적으로 구현되었는지 test
out=vgg16(a)
print(out)

tensor([[ 0.0018, -0.0014, -0.0019,  0.0119,  0.0191,  0.0202,  0.0025,  0.0059,
          0.0100, -0.0061]], grad_fn=<AddmmBackward>)


In [11]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(vgg16.parameters(), lr = 0.005, momentum = 0.9)

lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.9)
# epoch이 증가될때마다 learning rate을 점처 감소시켜서 정확도를 높이는 기법, 
# Step_size 5번마다 초기 lr에 0.9를 점차 곱해서 진행해라...라는 의미

Make Plot

In [13]:
# loss_plt = vis.line(Y=torch.Tensor(1).zero_(), 
#                    opts = dict (title = 'loss_tracker', legend = ['loss'], showlegend=True))

Training

In [ ]:
print(len(trainloader))
epochs = 1

for epoch in range (epochs):    # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = vgg16(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 30 ==29:      #print every 2000 mini-batches
#            loss_tracker(loss_plt, torch.Tensor         ######)
            print ('[%d, %5d] loss: %.3f'%
                  (epoch + 1, i+1, running_loss / 30))

    lr_sche.step()       # epoch 5회마다 lr에 0.9 곱하라...
    # In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.
    
print('Finished Training')

In [ ]:
# 학습된 상태를 model로 저장
# torch.save(vgg16.state_dict(),'./sample_data/vgg16_Trained_Model.pth')

In [27]:
# 앞서 학습된 모델을 불러서 loading 시키는 과정

vgg16.load_state_dict(torch.load('D:/Data/vgg16_epochs_70_Trained_Model.pth', map_location=torch.device('cpu')))
# colab GPU 학습 후, CPU로 loading하므로 map_location 옵션 사용함.

<All keys matched successfully>

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print Images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5' %classes[labels[j]] for j in range(4)))

In [31]:
outputs = vgg16(images.to(device))

In [34]:
_, predicted = torch.max(outputs, 1)
print('Predicted: ', ' '.join('%5s' %classes[predicted[j]] for j in range(4)))

Predicted:    cat  ship  ship plane


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = vgg16(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print ('Accuracy of the network on the 10000 test images: %d %%' % (100*correct/total))